# VAE

In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 
import numpy as np
import torch.nn.functional as F
import torchvision




In [2]:
train_data = datasets.MNIST("./datasets/", 
                            download=True, 
                            train=True, 
                            transform=transforms.Compose([
                                transforms.ToTensor()
                            ]))

test_data = datasets.MNIST('./datasets/', 
                           train=False, 
                           download=True, 
                           transform=transforms.Compose([
                               transforms.ToTensor()
                            ]))


In [3]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [4]:
x,y = iter(train_loader).next()
print(x.shape)

torch.Size([32, 1, 28, 28])


In [28]:
class Encoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Linear(128,64),
            nn.ReLU(inplace=True)
        )
        self.mu = nn.Linear(64, 20)
        self.logvar = nn.Linear(64, 20)
        
    def forward(self, x):
        h = self.encoder(x)
        return (self.mu(h), self.logvar(h))
    
class Decoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Linear(20,64),
            nn.ReLU(inplace=True),
            nn.Linear(64,256),
            nn.ReLU(inplace=True),
            nn.Linear(256,784),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.decoder(x)


class VAE(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def reparameterize(self, mu, logvar):
        # esp = (mu-mean_u)/sigma
        std = torch.exp(0.5*logvar)
        esp = torch.randn_like(std)
        return mu + std*esp
    
    def forward(self, x:torch.Tensor):
        x = x.flatten(1) # start_dim
        # x = self.encoder(x)
        ## 求mu logvar, 这里使用分解参数
        # mu, logvar = x.chunk(2, dim=1)
        mu, logvar = self.encoder(x)
        
        ## 参数重构，采样
        z = self.reparameterize(mu, logvar)
        x_hat = self.decoder(z)
        x_hat = x_hat.view(-1,1,28,28)
        return x_hat, mu, logvar
        
class VAELoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    
    def forward(self, x, x_hat, mu, logvar):
        # -1/2*(logvar - var^2 - mu +1)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        BCE = F.binary_cross_entropy(x_hat.view(-1, 784), x.view(-1, 784), reduction='sum')
        
        return BCE + KLD  

In [32]:
class VAE_1(nn.Module):
    def __init__(self):
        super(VAE_1, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [29]:
device = torch.device('cuda')
epochs = 10

In [35]:
# model = VAE(encoder=Encoder(), decoder=Decoder()).to(device)
model = VAE_1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criteon = VAELoss()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [37]:

for epoch in range(epochs):
    train_loss = 0
    for batch_idx, (x,y) in enumerate(train_loader):
        optimizer.zero_grad()
        x,y = x.to(device), y.to(device)
        x_hat, mu, logvar = model(x)
        loss = criteon(x, x_hat,mu, logvar)      
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(epoch, 'loss:', train_loss/ len(train_loader.dataset))
    # x, y = iter(test_loader).next()
    # with torch.no_grad():
    #     x_hat = model(x.to(device))
    # imshow(torchvision.utils.make_grid(x.cpu()))
    # imshow(torchvision.utils.make_grid(x_hat.cpu()))

    

0 loss: 6708723.850097656
1 loss: 6701523.369873047
2 loss: 6693227.812988281
3 loss: 6686384.433837891
4 loss: 6678336.035888672
5 loss: 6671787.381103516
6 loss: 6665827.294189453
7 loss: 6658363.057128906
8 loss: 6654258.286376953
9 loss: 6646871.090087891
